# Trying to adapt solution from [TFRS](https://www.tensorflow.org/recommenders/examples/basic_retrieval) tutorial for H&M data.

In [1]:
import os
import pprint
import tempfile
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

from typing import Dict, Text
from sklearn.model_selection import train_test_split

2023-10-27 18:28:28.274945: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-27 18:28:28.275005: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-27 18:28:28.280282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-27 18:28:28.712114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Short EDA on customer & article metadata

In [2]:
#article data
art_df = pd.read_csv('../../data/processed/articles_filled_priced.csv')

In [3]:
#customer data
cus_df = pd.read_csv('../../data/processed/customers_filled.csv')

- H&M transaction data were subsampled using [custom function](https://github.com/omegatro/IGP_2023/blob/main/modules/preprocessing.py to 5% of the original records (seed = 4312).
- The subsampled data were transformed to represent number of transactions for individual customer-product combinations (number of times each product was bought by each customer). The following code was used:
```
df = df.groupby(['customer_id', 'article_id']).count()
df = df[['customer_id', 'article_id','t_dat']].reset_index()
df.rename(columns={'t_dat':'t_count'}, inplace = True)
df.to_csv('0_05_4312_cus_art_grp_count.csv', header=True, index=False)
```
- The resulting file was uploaded to google drive to work with from colab.

In [4]:
#In this iteration, only ids are used as attributes for simplicity
counts = pd.read_csv('../../data/processed/0_05_4312_cus_art_grp_count.csv')[['customer_id', 'article_id']].sample(250_000)

In [5]:
#Adding product features
counts = counts.merge(art_df, left_on='article_id', right_on='article_id', how='left')

In [6]:
counts.columns

Index(['customer_id', 'article_id', 'product_code', 'prod_name',
       'product_type_no', 'product_type_name', 'product_group_name',
       'graphical_appearance_no', 'graphical_appearance_name',
       'colour_group_code', 'colour_group_name', 'perceived_colour_value_id',
       'perceived_colour_value_name', 'perceived_colour_master_id',
       'perceived_colour_master_name', 'department_no', 'department_name',
       'index_code', 'index_name', 'index_group_no', 'index_group_name',
       'section_no', 'section_name', 'garment_group_no', 'garment_group_name',
       'detail_desc', 'avg_price'],
      dtype='object')

In [7]:
#Adding customer features
counts = counts.merge(cus_df, left_on='customer_id', right_on='customer_id', how='left')

In [20]:
#Generating training and test subsets
train, test = train_test_split(counts, test_size=0.2)
#Getting unique user ids (required to transform those into embeddings by the model)
unq_cids = counts.customer_id.unique()

In [9]:
#Getting product ids (required to transform those into embeddings by the model)
articles = pd.read_csv('../../data/processed/articles_filled_priced.csv')[['article_id']]#.sample(sample_size, random_state=1)
unq_articles = articles.article_id.unique().astype(str)
article_ids = articles['article_id'].astype(str).map(lambda x: x.encode('utf-8'))

In [10]:
print(article_ids[:5])

0    b'108775015'
1    b'108775044'
2    b'108775051'
3    b'110065001'
4    b'110065002'
Name: article_id, dtype: object


In [11]:
print(unq_articles[:5])

['108775015' '108775044' '108775051' '110065001' '110065002']


In [12]:
len(unq_cids)

191869

In [13]:
len(unq_articles)

105542

# Converting from pandas df to tensorflow datasets for compatibility

In [20]:
train['customer_id'] = train.customer_id.astype(str)
train['article_id'] = train.article_id.astype(str)
train = train.to_dict(orient='list')
train = tf.data.Dataset.from_tensor_slices(train)

In [22]:
test['customer_id'] = test.customer_id.astype(str)
test['article_id'] = test.article_id.astype(str)
test = test.to_dict(orient='list')
test = tf.data.Dataset.from_tensor_slices(test)

In [23]:
for x in train.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'article_id': b'651724003',
 'customer_id': b'dccadcde4cec8a6ca4a4c018bf5a02c36ad9e35fba8e28c00114adc057a1'
                b'938c'}


In [24]:
for x in test.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'article_id': b'647355002',
 'customer_id': b'2920ade22c670d0ad69447ca93bd80dd6ee4854d98fa6cff9dc07afe12fe'
                b'ea73'}


# TFRS-based modeling experiments

# Adapting the model

In [25]:
embedding_dimension = 64

# Converting features into embedding vectors


In [26]:
customer_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unq_cids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.\
  tf.keras.layers.Embedding(len(unq_cids) + 1, embedding_dimension)
])

In [27]:
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unq_articles, mask_token=None),
  tf.keras.layers.Embedding(len(unq_articles) + 1, embedding_dimension)
])

In [28]:
#Generating embeddings from article ids to be used by evaluation layer to calculate the similarity score
article_embeddings = product_model(article_ids)
candidates_dataset = tf.data.Dataset.from_tensor_slices(article_embeddings)

In [29]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=candidates_dataset.batch(128)  # Batch the candidates for efficiency
)

In [30]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [31]:
class ProductlensModel(tfrs.Model):

  def __init__(self, customer_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.customer_model: tf.keras.Model = customer_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    customer_embeddings = self.customer_model(features["customer_id"])
    # And pick out the product features and pass them into the product model,
    # getting embeddings back.
    positive_product_embeddings = self.product_model(features["article_id"])

    # The task computes the loss and the metrics.
    return self.task(customer_embeddings, positive_product_embeddings, compute_metrics=not training)

In [ ]:
class NoBaseClassProductlensModel(tf.keras.Model):

  def __init__(self, customer_model, product_model):
    super().__init__()
    self.product_model: tf.keras.Model = product_model
    self.customer_model: tf.keras.Model = customer_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      customer_embeddings = self.customer_model(features["customer_id"])
      positive_product_embeddings = self.product_model(features["article_id"])
      loss = self.task(customer_embeddings, positive_product_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    customer_embeddings = self.customer_model(features["customer_id"])
    positive_product_embeddings = self.product_model(features["article_id"])
    loss = self.task(customer_embeddings, positive_product_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [32]:
#Hyperparameters
alpha = 25
batch_size = 256
epochs = 1

In [33]:
model = ProductlensModel(customer_model, product_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=alpha))

In [34]:
cached_train = train.batch(batch_size).cache()
cached_test = test.batch(batch_size).cache()

In [35]:
model.fit(cached_train, epochs=epochs)

2023-10-27 17:14:57.070422: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7c0881d6a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-27 17:14:57.070481: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5
2023-10-27 17:14:57.141631: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-27 17:14:57.664155: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-10-27 17:14:57.980266: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


  1/782 [..............................] - ETA: 1:07:02 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 1419.5621 - regularization_loss: 0.0000e+00 - total_loss: 1419.5621

2023-10-27 17:14:58.212368: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 [==============================] - 14s 12ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 727129.4247 - regularization_loss: 0.0000e+00 - total_loss: 727129.4247


In [36]:
model.evaluate(cached_test, return_dict=True)

196/196 [==============================] - 240s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.4539 - factorized_top_k/top_5_categorical_accuracy: 0.4543 - factorized_top_k/top_10_categorical_accuracy: 0.4545 - factorized_top_k/top_50_categorical_accuracy: 0.4551 - factorized_top_k/top_100_categorical_accuracy: 0.4553 - loss: 1301809.2741 - regularization_loss: 0.0000e+00 - total_loss: 1301809.2741


{'factorized_top_k/top_1_categorical_accuracy': 0.4538800120353699,
 'factorized_top_k/top_5_categorical_accuracy': 0.45427998900413513,
 'factorized_top_k/top_10_categorical_accuracy': 0.45451998710632324,
 'factorized_top_k/top_50_categorical_accuracy': 0.45513999462127686,
 'factorized_top_k/top_100_categorical_accuracy': 0.4553399980068207,
 'loss': 358053.0625,
 'regularization_loss': 0,
 'total_loss': 358053.0625}

In [37]:
# If 'test' is a dataset of dictionaries that include 'article_id'
candidates = test.map(lambda x: (x['article_id'], model.product_model(x['article_id'])))

index = tfrs.layers.factorized_top_k.BruteForce(model.customer_model)
index.index_from_dataset(candidates.batch(100))

In [38]:
# Get recommendations for a specific customer
customer_id = 'abc'
_, titles = index(tf.constant([customer_id]))
print(f"Recommendations for customer {customer_id}: {titles[0].numpy().astype(str).tolist()}")

Recommendations for customer abc: ['854384005', '854384005', '854384005', '357792011', '775482001', '335037001', '823063002', '823063002', '682771001', '682771001']


# Ranking model

In [15]:
#In this iteration, only ids are used as attributes for simplicity
counts = pd.read_csv('../../data/processed/0_05_4312_cus_art_grp_count.csv')[['customer_id', 'article_id','t_count']]
#Generating training and test subsets
train, test = train_test_split(counts, test_size=0.2)
#Getting unique user ids (required to transform those into embeddings by the model)
unq_cids = counts.customer_id.unique()

In [18]:
#Getting product ids (required to transform those into embeddings by the model)
articles = pd.read_csv('../../data/processed/articles_filled_priced.csv')[['article_id']]
unq_articles = articles.article_id.unique().astype(str)
article_ids = articles['article_id'].astype(str).map(lambda x: x.encode('utf-8'))

In [21]:
train['customer_id'] = train.customer_id.astype(str)
train['article_id'] = train.article_id.astype(str)
train = train.to_dict(orient='list')
train = tf.data.Dataset.from_tensor_slices(train)

In [22]:
for x in train.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'article_id': b'714790003',
 'customer_id': b'39bc5e71c3609f8e37bd24099d7793dabbd9fdd73261e443f0a929e79b7e'
                b'658e',
 't_count': 1}


In [23]:
test['customer_id'] = test.customer_id.astype(str)
test['article_id'] = test.article_id.astype(str)
test = test.to_dict(orient='list')
test = tf.data.Dataset.from_tensor_slices(test)

In [24]:
for x in test.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'article_id': b'768759002',
 'customer_id': b'99b8cd105ee382f5c0bdaf672696522a0564eec294e94b11ac8b8d7bd2d1'
                b'3366',
 't_count': 1}


In [25]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.customer_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unq_cids , mask_token=None),
      tf.keras.layers.Embedding(len(unq_cids) + 1, embedding_dimension)
    ])

    # Compute embeddings for product.
    self.product_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unq_articles, mask_token=None),
      tf.keras.layers.Embedding(len(unq_articles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    cus_id, art_id = inputs

    cus_embedding = self.customer_embeddings(cus_id)
    art_embedding = self.product_embeddings(art_id)

    return self.ratings(tf.concat([cus_embedding, art_embedding], axis=1))

In [26]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [27]:
class ProductlensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["customer_id"], features["article_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("t_count")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [43]:
#Hyperparameters
alpha = 0.001
batch_size = 128
epochs = 5

In [44]:
model = ProductlensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=alpha))

In [45]:
cached_train = train.batch(batch_size).cache()
cached_test = test.batch(batch_size).cache()

In [ ]:
model.fit(cached_train, epochs=epochs)

Epoch 1/5
4102/9833 [===========>..................] - ETA: 45s - root_mean_squared_error: 0.2422 - loss: 0.0587 - regularization_loss: 0.0000e+00 - total_loss: 0.0587

In [32]:
model.evaluate(cached_test, return_dict=True)

2459/2459 [==============================] - 11s 4ms/step - root_mean_squared_error: 0.1122 - loss: 0.0126 - regularization_loss: 0.0000e+00 - total_loss: 0.0126


{'root_mean_squared_error': 0.11222855001688004,
 'loss': 2.9352251658565365e-05,
 'regularization_loss': 0,
 'total_loss': 2.9352251658565365e-05}

In [33]:
test_counts = {}
sample = counts.sample(1000)
test_product_ids = zip(sample['customer_id'],sample['article_id'], sample['t_count'])
for pid, aid, t_count in test_product_ids:
  test_counts[pid] = (model({
      "customer_id": np.array(['73f7d8cca33b8236879d92a6188f691410a82763dc8bb65b8829568c07f8d9a7']),
      "article_id": np.array([pid])

  }).numpy()[0,0], t_count)

In [34]:
print("t_counts:")
for title, score in sorted(test_counts.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

t_counts:
834a961effec223096b1dde1ae488f173af3f6d85b0d9123b3d0017549aa289f: (1.0046736, 2)
9ee1d750132abb253fc8767b7e195befa5431628274712e208bce5ab5becf27a: (1.0046736, 2)
3bb0c14530e99a29bf8bc431e159e37ae4bcc70b282ee470e6d4118d80fe5687: (1.0046736, 2)
65cb62c794232651e2ac711faa11c2b4e3d41d5f3b59b50bee3ffde1d5776644: (1.0046736, 2)
ec2b77deb3e424c703326af0452162d9a44a5a593676be656d99a1b021d55785: (1.0046736, 2)
5c74f5d8682b182ead8b246746927fd78cb4335f306d0e63761c403dcaa4586b: (1.0046736, 2)
5f3005e5c329357ef45ea12191abb39c388d6f04052ce7f410ed2ba3e19f9670: (1.0046736, 2)
8e545f85ba659e44e41231802da53deec93e01ed61c41beb5db393d91dca1c65: (1.0046736, 2)
b327d23897ea1f0aff088a11d20c904b96598e757b40d049a4a882b03f7ee60a: (1.0046736, 2)
dd63b4a61b4bc1a91d88e4552fd9648c1747fdd827fb2ae765040459aabae493: (1.0046736, 2)
5f8ccf30a8ad0b124a7f78560b9f7cba4470a55003263f20881dacb6cc34b889: (1.0046736, 2)
93b849f5b290350dfd85d75d80a7dea4758ec9bf784117765f2dec423c8fa515: (1.0046736, 2)
4f71521d25f4fbcca6

# Including Features

In [ ]:
embedding_dimension = 32
unq_cids = counts.customer_id.unique()
unq_articles = articles.article_id.unique().astype(str)
unq_prod_names = pd.read_csv('../data/articles.csv').prod_name.unique().astype(str)

In [ ]:
customer_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unq_cids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.\
  tf.keras.layers.Embedding(len(unq_cids) + 1, embedding_dimension)
])

In [ ]:
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unq_articles, mask_token=None),
  tf.keras.layers.Embedding(len(unq_cids) + 1, embedding_dimension)
])

In [ ]:
product_name_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unq_prod_names, mask_token=None),
  tf.keras.layers.Embedding(len(unq_articles) + 1, embedding_dimension)
])

In [ ]:
class CustomerModel(tf.keras.Model):

  def __init__(self, customer_model):
    super().__init__()

    self.customer_embedding = customer_model

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return self.customer_embedding(inputs['customer_id'])


In [ ]:
class ProductModel(tf.keras.Model):

  def __init__(self, product_model):
    super().__init__()

    self.product_embedding = product_model
    self.prod_name_embedding = product_name_model

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([self.product_embedding(inputs['article_id']), self.prod_name_embedding(inputs['prod_name'])], axis = 1)

In [ ]:
for t in train.map(lambda x: {'cid':x['customer_id'], 'FN':x['FN']}).batch(1).take(1):
  print(t)

In [ ]:
cus_model = CustomerModel(customer_model)
result = cus_model.predict(train.map(lambda x:{'customer_id': x['customer_id']}).batch(1).take(1).as_numpy_iterator())
print(f"Computed representations: {result}")

In [ ]:
prod_model = ProductModel(product_model)
result = prod_model.predict(train.map(lambda x:{'article_id': x['article_id'], 'prod_name':x['prod_name']}).batch(1).take(1).as_numpy_iterator())
print(f"Computed representations: {result}")

In [ ]:
#Generating embeddings from article ids to be used by evaluation layer to calculate the similarity score
articles = pd.read_csv('../data/articles.csv')[['article_id', 'prod_name']]#.sample(sample_size, random_state=1)
article_ids = articles.astype(str).map(lambda x: x.encode('utf-8'))
article_embeddings = prod_model(article_ids)
candidates_dataset = tf.data.Dataset.from_tensor_slices(article_embeddings)

In [ ]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries (customers)."""

  def __init__(self, layer_sizes, customer_model):
    """Model for encoding user queries (customers).

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = CustomerModel(customer_model)

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class CandidateModel(tf.keras.Model):
  """Model for encoding products."""

  def __init__(self, layer_sizes, product_model):
    """Model for encoding products.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = ProductModel(product_model)

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [ ]:
class ProductlensModel(tfrs.models.Model):

  def __init__(self, layer_sizes, product_model, customer_model):
    super().__init__()
    self.query_model = QueryModel(layer_sizes, customer_model)
    self.candidate_model = CandidateModel(layer_sizes, product_model)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=train.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "customer_id": features["customer_id"]
    })
    product_embeddings = self.candidate_model({'article_id':features["article_id"], "prod_name":features["prod_name"]})

    return self.task(
        query_embeddings, product_embeddings, compute_metrics=not training)

In [ ]:
num_epochs = 100
learning_rate = 0.5
batch_size=1024

In [ ]:
cached_train = train.batch(batch_size).cache()
cached_test = test.batch(batch_size).cache()

In [ ]:
model = ProductlensModel([128, 64, 32], product_model=product_model, customer_model=customer_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate))

In [ ]:
three_layer_history = model.fit(
    cached_train,
    epochs=num_epochs,
    verbose=1)


In [ ]:
model.evaluate(cached_test, return_dict=True)